In [1]:
import pandas as pd
import numpy
from __future__ import division

In [2]:
df1_WS2T = pd.read_csv("/Users/asteriag/Desktop/DATASCIENCEPROJECT_WORKFLOW/pythonProject/segment1_WS2_has_duration.csv")

In [3]:
df1_WS2T.head(n=1)

,CASE_ID,WRKFLW_TSK_MAP_GROUPED,WRKFLW_TSK_MAP_DETAILED,PATH_CRDATETIME,ITEM_CRUSER,WRKFLW_ORG_TYPE_CRUSER,STATUS_ORDER,LATEST_STATUS,WRKFLW_PROCESS,QUERY_PART,...,SPK_CAT_TYPE_CD,BROAD_COURSE_LEVEL,DEGREETYPE,LIABILITY_CATEGORY_CD_RECODE,ORIG_LIABILITY_CATEGORY_CD_RECODE,OFFER_RELEASED_FG,OFFER_OUTCOME_CD,OFFER_CONDITIONS,time_used_str,time_used_second
0,1172679|570743|869833,Send for Assessment,Send for Assessment,24/1/2020 15:09:59,1172679,NaN,1,Y,1. App to Assessment,Part 1 - Application ID Workflow,...,MCW,HDCW,Full Degree,OF,OF,NaN,NaN,Y,NaN,0.0


In [4]:
numOfWorkflowProcesses = df1_WS2T.groupby('CASE_ID').CASE_ID.count()

count_dic=dict()
for i,v in numOfWorkflowProcesses.items():
    if count_dic.get(v) is None:
        count_dic[v]=[]
        count_dic[v].append(i)
    else:
        count_dic[v].append(i)
#count_dic


In [5]:
len_list=[]
for key in count_dic.keys():
    len_list.append(len(count_dic[key]))

In [6]:
def strfdelta(tdelta, fmt):
    d=dict()
    d["days"],rem1 = divmod(tdelta,60*60*24)
    d["hours"], rem2 = divmod(rem1, 3600)
    d["minutes"], d["seconds"] = divmod(rem2, 60)
    return fmt.format(**d)

In [7]:
list_avgtime_per_key =[]
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_person = 0
    for id in id_list:
        df = df1_WS2T[df1_WS2T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_person+=tsum
    list_avgtime_per_key.append(int((list_sumtime_person/len(count_dic[key]))))

list_avgtime_per_key_str = []
for i in list_avgtime_per_key:
    list_avgtime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [8]:
list_mediantime_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_sumtime_per_person = []
    for id in id_list:
        df = df1_WS2T[df1_WS2T['CASE_ID']== id]
        tsum = df['time_used_second'].sum()
        list_sumtime_per_person.append(tsum)
    list_mediantime_per_key.append(numpy.median(list_sumtime_per_person))

list_mediantime_per_key_str = []
for i in list_mediantime_per_key:
    list_mediantime_per_key_str.append(strfdelta(i,"{days} days {hours} hours {minutes} minutes {seconds} seconds"))

In [9]:
list_percent_reach_end = []
for key in count_dic.keys():
    id_list = count_dic[key]
    count_df_end_step = 0
    count_df_id = 0
    for id in id_list:
        df_id = df1_WS2T[df1_WS2T['CASE_ID']== id]
        df_end_step = df_id[df_id['WRKFLW_TSK_MAP_GROUPED'].isin(['Assessment Outcome - Successful','Assessment Outcome - Not Successful']) ]
        count_df_id += df_id['CASE_ID'].count()
        count_df_end_step += df_end_step['CASE_ID'].count()
    list_percent_reach_end.append(str(round((count_df_end_step/count_df_id)*100,2))+ "%")

In [10]:
list_step_per_key = []
for key in count_dic.keys():
    id_list = count_dic[key]
    list_step = set()
    for id in id_list:
        df = df1_WS2T[df1_WS2T['CASE_ID']== id]
        for index, row in df.iterrows():
            list_step.add(row['WRKFLW_TSK_MAP_GROUPED'])
    list_step_per_key.append(list_step)

In [11]:
s1_t2=pd.DataFrame({'Number of Workflow processes in this Workflow Segment':count_dic.keys(),'Number of cases':len_list, 'average time for workflow segment':list_avgtime_per_key_str,'median time for workflow segment':list_mediantime_per_key_str,'percentage of cases reaching to end of workflow segment':list_percent_reach_end,'step':list_step_per_key})
s1_t2.head(n=10)


,Number of Workflow processes in this Workflow Segment,Number of cases,average time for workflow segment,median time for workflow segment,percentage of cases reaching to end of workflow segment,step
0,1,835,0 days 0 hours 0 minutes 0 seconds,0.0 days 0.0 hours 0.0 minutes 0.0 seconds,0.0%,{Send for Assessment}
1,2,861,26 days 21 hours 51 minutes 27 seconds,15.0 days 18.0 hours 14.0 minutes 28.0 seconds,31.3%,"{OC - Reviewed - Other, Assessment Outcome - H..."
2,3,2852,30 days 12 hours 57 minutes 59 seconds,18.0 days 20.0 hours 40.0 minutes 0.0 seconds,30.39%,"{OC - Reviewed - Other, Assessment Outcome - H..."
3,4,2205,39 days 20 hours 51 minutes 7 seconds,27.0 days 14.0 hours 54.0 minutes 30.0 seconds,23.05%,"{OC - Reviewed - Other, Assessment Outcome - H..."
4,5,1595,45 days 18 hours 11 minutes 27 seconds,30.0 days 0.0 hours 12.0 minutes 29.0 seconds,18.26%,"{OC - Reviewed - Other, Assessment Outcome - H..."
5,6,790,52 days 18 hours 20 minutes 54 seconds,36.0 days 21.0 hours 8.0 minutes 1.0 seconds,15.04%,"{OC - Reviewed - Other, Assessment Outcome - H..."
6,7,443,66 days 17 hours 1 minutes 56 seconds,44.0 days 19.0 hours 51.0 minutes 54.0 seconds,11.58%,"{OC - Reviewed - Other, Assessment Outcome - H..."
7,8,240,63 days 2 hours 21 minutes 41 seconds,41.0 days 4.0 hours 16.0 minutes 3.0 seconds,10.89%,"{OC - Reviewed - Other, Assessment Outcome - H..."
8,9,161,59 days 4 hours 29 minutes 9 seconds,41.0 days 23.0 hours 43.0 minutes 14.0 seconds,9.73%,"{OC - Reviewed - Other, Assessment Outcome - H..."
9,10,127,84 days 18 hours 22 minutes 14 seconds,47.0 days 22.0 hours 27.0 minutes 26.0 seconds,8.66%,"{OC - Reviewed - Other, Assessment Outcome - H..."


In [12]:
s1_t2.to_csv('segment1_WS2_table.csv', index=False)









